# LTV Heatmap
Have to learn how to make an LTV Heatmap using Cohort analysis?

In [1]:
# Libraries
import pandas as pd

In [2]:
# load data
df = pd.read_csv('payments.csv')

In [3]:
df.head()

,account_id,payment_id,payment_usd,payment_date
0,hbQ5GR1L54QtQAB,QhQfR06N8dQAO,50.00,2021-01-28
1,vsQ5GR1KF4QtQAH,Q7QfR06N8dQAO,968.07,2021-01-28
2,5jQ5GR1Uz4QtQAB,Q6QfR06N8dQAO,763.67,2021-01-28
3,hnQ5GR1O34QtQAB,Q2QfR06N8dQAO,2378.57,2021-01-28
4,ztQ5GR1Ha4QtQAF,Q0QfR06N8dQAO,2855.84,2021-01-28


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6890 entries, 0 to 6889
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   account_id    6890 non-null   object 
 1   payment_id    6890 non-null   object 
 2   payment_usd   6890 non-null   float64
 3   payment_date  6890 non-null   object 
dtypes: float64(1), object(3)
memory usage: 215.4+ KB


In [8]:
user_counts = pd.value_counts(df['account_id'])
user_counts.head()

xqQ5GR06N4QtQAC    28
GzQ5GR1Fn4QtQAJ    27
nUQ5GR1EK4QtQAT    25
RcQ5GR0p44QtQAI    24
cwQ5GR15j4QtQAE    24
Name: account_id, dtype: int64

In [9]:
pd.value_counts(df['payment_id'])

CBQfR02t8dQA2    2
BtQfR02t8dQAM    2
AmQfR02t8dQAM    2
i4QfR05g8dQAA    2
i5QfR05g8dQAA    2
                ..
hwQfR0638dQAE    1
hxQfR0638dQAE    1
hyQfR0638dQAE    1
hzQfR0638dQAE    1
XyQfR05K8dQAK    1
Name: payment_id, Length: 6611, dtype: int64

In [10]:
df.loc[df['payment_id']=='CBQfR02t8dQA2']

,account_id,payment_id,payment_usd,payment_date
3595,sqQ5GR1T34QtQAB,CBQfR02t8dQA2,125.00,2020-09-22
3727,wxQ5GR06N4QtQAC,CBQfR02t8dQA2,1083.68,2020-09-16


Weird....some of the payment IDs are duplicates, but theyre associated with different accounts?


Really need to learn wtf is going on here. 

In [13]:
# Lets create a dataframe with only each accounts first purchase
df_sorted_dates = df.sort_values(by='payment_date') # sort in ascending by payment date
accounts_unique = df_sorted_dates.drop_duplicates(subset=['account_id'], keep='first')

In [19]:
df.loc[df['account_id']=='xqQ5GR06N4QtQAC']

,account_id,payment_id,payment_usd,payment_date
604,xqQ5GR06N4QtQAC,7nQfR06N8dQAG,1981.13,2021-01-08
817,xqQ5GR06N4QtQAC,hnQfR06N8dQAC,125.00,2021-01-01
1496,xqQ5GR06N4QtQAC,TBQfR06N8dQA0,1581.51,2020-12-07
1742,xqQ5GR06N4QtQAC,TAQfR06N8dQA0,125.00,2020-12-01
1760,xqQ5GR06N4QtQAC,0HQfR06M8dQAS,125.00,2020-12-01
2389,xqQ5GR06N4QtQAC,u9QfR0638dQAE,1070.73,2020-11-07
2483,xqQ5GR06N4QtQAC,teQfR0638dQAE,125.00,2020-11-06
2583,xqQ5GR06N4QtQAC,7hQfR0638dQAE,250.00,2020-11-01
3213,xqQ5GR06N4QtQAC,KjQfR02t8dQAM,841.16,2020-10-07
3264,xqQ5GR06N4QtQAC,i3QfR02t8dQAE,125.00,2020-10-05
